# PROJETO DE CLASSIFICACAO - COMPARACAO DE ALGORITMOS

 Aluno: Davi Medeiros 
 Data: 28/11/2025

# 1 IMPORTACAO DE BIBLIOTECAS

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("PROJETO DE CLASSIFICACAO - CREDIT SCORING")
print("=" * 60)

# 2 CRIACAO DA BASE DE DADOS

In [ ]:
def create_credit_data():
    """
    Cria dataset simulado de credit scoring com 1000 observacoes
    """
    np.random.seed(42)
    n_samples = 1000
    
    data = {
        'idade': np.random.randint(18, 70, n_samples),
        'renda_mensal': np.random.normal(5000, 2000, n_samples),
        'valor_emprestimo': np.random.normal(15000, 8000, n_samples),
        'prazo_emprestimo': np.random.randint(6, 60, n_samples),
        'historico_credito': np.random.choice(['bom', 'regular', 'ruim'], n_samples, p=[0.6, 0.3, 0.1]),
        'emprego_tipo': np.random.choice(['assalariado', 'autonomo', 'empresario', 'desempregado'], n_samples, p=[0.5, 0.3, 0.15, 0.05]),
        'tempo_emprego': np.random.uniform(0, 20, n_samples),
        'dependentes': np.random.randint(0, 5, n_samples),
        'score_serasa': np.random.randint(0, 1000, n_samples)
    }

In [ ]:
 df = pd.DataFrame(data)
    
    # Criar variavel target (inadimplente: 0 = nao, 1 = sim)
    prob_inadimplente = (
        (df['historico_credito'] == 'ruim') * 0.6 +
        (df['historico_credito'] == 'regular') * 0.3 +
        (df['score_serasa'] < 300) * 0.4 +
        (df['renda_mensal'] < 2000) * 0.5 +
        (df['valor_emprestimo'] / df['renda_mensal'] > 10) * 0.4 +
        np.random.normal(0, 0.1, n_samples)
    )
    
    df['inadimplente'] = (prob_inadimplente > 0.5).astype(int)
    
    return df

# Criar dataset
df = create_credit_data()
print("Base de Dados de Credit Scoring")
print(f"Shape: {df.shape}")
print(f"Distribuicao da variavel target:")
print(df['inadimplente'].value_counts())
print(f"Proporcao: {df['inadimplente'].value_counts(normalize=True)}")


# 3 PRE-PROCESSAMENTO DOS DADOS

In [ ]:
def preprocess_credit_data(df):
    """
    Realiza pre-processamento dos dados
    """
    data = df.copy()
    
    # Verificar valores missing
    print("Valores missing por coluna:")
    print(data.isnull().sum())
    
    # Separar variaveis preditoras e target
    X = data.drop('inadimplente', axis=1)
    y = data['inadimplente']
    
    return X, y

X, y = preprocess_credit_data(df)
print(f"Pre-processamento concluido")
print(f"Variaveis preditoras: {X.shape[1]}")
print(f"Target: {y.shape[0]} observacoes")

# 4 CODIFICACAO DE VARIAVEIS CATEGORICAS

In [ ]:
def encode_categorical_features(X):
    """
    Aplica LabelEncoder para variaveis categoricas
    """
    X_encoded = X.copy()
    
    # LabelEncoder para variaveis categoricas
    label_encoders = {}
    categorical_columns = ['historico_credito', 'emprego_tipo']
    
    for col in categorical_columns:
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col])
        label_encoders[col] = le
        print(f"{col} codificado: {dict(zip(le.classes_, le.transform(le.classes_)))}")
    
    return X_encoded, label_encoders

X_encoded, label_encoders = encode_categorical_features(X)
print(f"Shape apos codificacao: {X_encoded.shape}")

# 5 PADRONIZACAO DE VARIAVEIS NUMERICAS

In [ ]:
def standardize_numeric_features(X):
    """
    Aplica StandardScaler para variaveis numericas
    """
    scaler = StandardScaler()
    
    # Colunas numericas para padronizar
    numeric_cols = ['idade', 'renda_mensal', 'valor_emprestimo', 'prazo_emprestimo', 
                   'tempo_emprego', 'dependentes', 'score_serasa']
    
    X_standardized = X.copy()
    X_standardized[numeric_cols] = scaler.fit_transform(X_standardized[numeric_cols])
    
    return X_standardized, scaler

X_standardized, scaler = standardize_numeric_features(X_encoded)
print("Padronizacao concluida")

# 6 DIVISAO DOS DADOS E SALVAMENTO

In [ ]:
# Dividir entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_standardized, y, test_size=0.3, random_state=42, stratify=y
)

print("Divisao dos dados:")
print(f"Treino: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Teste:  X_test {X_test.shape}, y_test {y_test.shape}")

# Salvar os datasets em arquivos pkl
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

print("Arquivos salvos com sucesso!")

# 7 TREINAMENTO DOS MODELOS

In [ ]:
# Definir e treinar multiplos algoritmos
models = {
    'Naive Bayes': GaussianNB(),
    'Arvore de Decisao': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
    'KNN (k=7)': KNeighborsClassifier(n_neighbors=7)
}

# Dicionario para armazenar resultados
results = {}

print("Treinando modelos...")
for name, model in models.items():
    print(f"Treinando {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'f1_score': f1,
        'predictions': y_pred
    }
    
    print(f"   {name} - Acuracia: {accuracy:.4f}, F1-Score: {f1:.4f}")

# Salvar modelos treinados
for name, result in results.items():
    with open(f'modelo_{name.replace(" ", "_").replace("(", "").replace(")", "")}.pkl', 'wb') as f:
        pickle.dump(result['model'], f)

print("Modelos salvos em arquivos .pkl")


# 8 COMPARACAO DE PERFORMANCE

In [ ]:
# Criar relatorio comparativo
print("COMPARACAO DOS ALGORITMOS")
print("=" * 60)

comparison_df = pd.DataFrame({
    'Modelo': list(results.keys()),
    'Acuracia': [results[name]['accuracy'] for name in results.keys()],
    'F1-Score': [results[name]['f1_score'] for name in results.keys()]
}).sort_values('Acuracia', ascending=False)

print(comparison_df.round(4))
print("=" * 60)

# Identificar melhor modelo
best_model_name = comparison_df.iloc[0]['Modelo']
best_accuracy = comparison_df.iloc[0]['Acuracia']
best_f1 = comparison_df.iloc[0]['F1-Score']

print(f"MELHOR MODELO: {best_model_name}")
print(f"   Acuracia: {best_accuracy:.4f}")
print(f"   F1-Score: {best_f1:.4f}")

# Detalhes do melhor modelo
print(f"Relatorio detalhado do {best_model_name}:")
print(classification_report(y_test, results[best_model_name]['predictions']))

# Matriz de confusao do melhor modelo
print(f"Matriz de Confusao - {best_model_name}:")
conf_matrix = confusion_matrix(y_test, results[best_model_name]['predictions'])
print(conf_matrix)

# 9 VISUALIZACAO DOS RESULTADOS

In [ ]:
# Configurar estilo dos graficos
plt.style.use('default')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Grafico 1: Comparacao de Acuracia
models_names = comparison_df['Modelo']
accuracies = comparison_df['Acuracia']
colors = ['green' if x == best_model_name else 'skyblue' for x in models_names]

ax1.barh(models_names, accuracies, color=colors)
ax1.set_xlabel('Acuracia')
ax1.set_title('Comparacao de Acuracia dos Modelos')
ax1.set_xlim(0, 1)

# Grafico 2: Comparacao de F1-Score
f1_scores = comparison_df['F1-Score']
colors_f1 = ['orange' if x == best_model_name else 'lightcoral' for x in models_names]

ax2.barh(models_names, f1_scores, color=colors_f1)
ax2.set_xlabel('F1-Score')
ax2.set_title('Comparacao de F1-Score dos Modelos')
ax2.set_xlim(0, 1)

# Grafico 3: Matriz de Confusao do Melhor Modelo
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax3)
ax3.set_title(f'Matriz de Confusao - {best_model_name}')
ax3.set_ylabel('Verdadeiro')
ax3.set_xlabel('Predito')

# Grafico 4: Importancia das Features (apenas para Random Forest)
if best_model_name == 'Random Forest':
    feature_importance = results[best_model_name]['model'].feature_importances_
    features = X_train.columns
    feature_df = pd.DataFrame({'feature': features, 'importance': feature_importance})
    feature_df = feature_df.sort_values('importance', ascending=True)
    
    ax4.barh(feature_df['feature'], feature_df['importance'])
    ax4.set_title('Importancia das Features - Random Forest')
    ax4.set_xlabel('Importancia')

plt.tight_layout()
plt.savefig('comparacao_modelos.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("PROJETO CONCLUIDO COM SUCESSO!")
print("ARQUIVOS GERADOS:")
print("   - X_train.pkl, X_test.pkl, y_train.pkl, y_test.pkl")
print("   - modelo_[NOME_DO_MODELO].pkl para cada algoritmo")
print("   - comparacao_modelos.png (graficos comparativos)")